In [ ]:
import requests
import random
import time
import pandas as pd

from GPMLoginAPI import GPMLoginAPI
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from UndetectChromeDriver import UndetectChromeDriver


In [ ]:
STATUS = { 0: "Buy It Now",
          38: "Place Bid",
          33: "Make Offer",
          39: "Buy It Now",
          23: "Buy It Now",
          10: "Get It",
          15: "Get It"}

In [ ]:
HEADERS = {"Accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
           "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}


In [ ]:
domains = open('domain_go.txt').read().splitlines()
len(domains)

In [ ]:
# API GPM
api = GPMLoginAPI('http://127.0.0.1:19995')

In [ ]:
# Tạo proifile GPM
def init_driver():
    profileid = api.Create(time.time())['profile_id']
    startedResult = api.Start(profileid)
    browserLocation = str(startedResult["browser_location"])
    seleniumRemoteDebugAddress = str(startedResult["selenium_remote_debug_address"])
    gpmDriverPath = str(startedResult["selenium_driver_location"])
    options = Options()
    options.debugger_address = seleniumRemoteDebugAddress
    myService  = service.Service(gpmDriverPath)
    return UndetectChromeDriver(service = myService, options=options), profileid

In [ ]:
# Load cookies
def reload_cookies():
    driver.get(f'https://www.godaddy.com/domainsearch/find?checkAvail=1&domainToCheck={random.choice(domains)}')
    # Có thể optimize bằng chờ load element thay vì mặc định 10s
    time.sleep(10)
    return {"bm_sz": driver.get_cookie("bm_sz")['value'],
          "visitor": driver.get_cookie("visitor")['value'],
          "market": "en-US"}

In [ ]:
# Proxies Debug
proxies = {'http': 'http://192.168.113.6:1234',
           'https': 'http://192.168.113.6:1234'}

In [ ]:
# Data lưu list json dự liệu
data = []
# Biến đếm lỗi.
c = 0
# Khởi tạo GPM
driver, profile_id = init_driver()
# Lấy cookies
cookies = reload_cookies()

In [ ]:
for domain in domains:
    # Nếu domain đã get bỏ qua
    if domain in [domain['ClientRequestIn']['DomainName'] for domain in data]:
        continue
    while True:
        try:
            r = requests.get(f'https://www.godaddy.com/domainfind/v1/search/exact?req_id={int(time.time() * 1000)}&key=dpp_search&q={domain}', cookies=cookies, headers=HEADERS)
            print(r.status_code)
            # Hạn chế IP
            if r.status_code == 429:
                print('IP')
                raise
            # Hạn chế Session và Browser
            elif r.status_code == 403:
                c += 1
                # Lỗi 403 quá 3 lần thay profile GPM
                if c >= 3:
                    driver.close()
                    api.Delete(profile_id)
                    driver, profile = init_driver()
                    c = 0
                # Refresh lại cookies
                cookies = reload_cookies()
            # Lấy dự liệu thành công
            elif r.status_code == 200:
                data.append(r.json())
                c = 0
                print(f'{domain} Done!')
                break
        except Exception as e:
            print(c, e)


In [ ]:

""" 
Xử lý dữ liệu
Có 2 dạng url:
Get It
https://auctions.godaddy.com/trpItemListing.aspx?src=dpp&miid={AuctionId}&itc=dpp_absol1
Make Offer
https://www.godaddy.com/domain-auctions/{domain thay '.' thành '-'}-{AuctionId}?src=dpp&itc=dpp_absol1
"""


In [ ]:
final_data = []
for data_ in data:
    domain_name = data_['ClientRequestIn']['DomainName']
    price = data_['ExactMatchDomain']['PriceDisplay']
    status = STATUS[data_['ExactMatchDomain']['AuctionTypeId']] if price else "Taken"
    auctionid = data_['ExactMatchDomain']['AuctionId']
    if status == 'Get It':
        url = f'https://auctions.godaddy.com/trpItemListing.aspx?src=dpp&miid={auctionid}&itc=dpp_absol1'
    elif status == 'Make Offer':
        url = f"https://www.godaddy.com/domain-auctions/{domain_name.replace('.', '-')}-{auctionid}?src=dpp&itc=dpp_absol1"
    else:
        url = ""
    final_data.append((domain_name, status, price, url))


In [ ]:
pd.DataFrame(final_data, columns = ['Domain Name', 'Status', 'Price', 'URL']).to_csv('ouput.csv')